In [9]:
# Cell 1: imports and display options

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

pd.set_option("display.width", 140)
pd.set_option("display.max_columns", 30)


In [13]:
# Cell 2

df = pd.read_pickle("/Users/prithvivarman/Desktop/Projects/TWSQ DataProject 2/DataProject.pk")

# Ensure date is datetime and sorted
df["date"] = pd.to_datetime(df["date"])
df = df.sort_values("date")

print(df.head())
print("\nItems:", df["item"].unique())
print("Assets:", df["asset"].unique())

# --- Pivot each item separately ---

# Returns
ret_raw = (
    df[df["item"] == "daily_ret"]
    .pivot_table(index="date", columns="asset", values="value")
)

# Highs
high_raw = (
    df[df["item"] == "daily_high"]
    .pivot_table(index="date", columns="asset", values="value")
)

# Lows
low_raw = (
    df[df["item"] == "daily_low"]
    .pivot_table(index="date", columns="asset", values="value")
)

print("\nShapes:")
print("ret_raw :", ret_raw.shape)
print("high_raw:", high_raw.shape)
print("low_raw :", low_raw.shape)


           date     asset       value        item
0    2020-01-01    ADAUSD    0.033813  daily_high
1450 2020-01-01  LINK-USD -999.000000   daily_ret
1449 2020-01-01   ETH-USD -999.000000   daily_ret
1448 2020-01-01   BTC-USD -999.000000   daily_ret
2895 2020-01-01   ADA-USD    0.032704   daily_low

Items: ['daily_high' 'daily_ret' 'daily_low']
Assets: ['ADAUSD' 'LINK-USD' 'ETH-USD' 'BTC-USD' 'ADA-USD' 'LINKUSD' 'ETHUSD'
 'BTCUSD']

Shapes:
ret_raw : (362, 4)
high_raw: (362, 4)
low_raw : (271, 4)


In [15]:
# Cell 3: clean data

# 3.1 Align high and low so indices & columns match
high_aligned, low_aligned = high_raw.align(low_raw, join="inner")

print("Aligned high shape:", high_aligned.shape)
print("Aligned low  shape:", low_aligned.shape)

# 3.2 Sanity check: high should not be below low
bad_mask = (high_aligned < low_aligned)
bad_dates = bad_mask.any(axis=1)

print("Number of dates with high < low:", bad_dates.sum())
if bad_dates.sum() > 0:
    print("Example problematic dates:", bad_dates[bad_dates].index[:5])

# Drop bad dates from all series (returns + high + low)
good_dates = high_aligned.index[~bad_dates]
ret_clean = ret_raw.loc[good_dates].copy()

# 3.3 Drop dates with any missing returns (avoid partial cross-section)
ret_clean = ret_clean.dropna(how="any")

print("\nClean return matrix shape:", ret_clean.shape)
print("Clean date range:", ret_clean.index.min(), "to", ret_clean.index.max())
print("Assets:", list(ret_clean.columns))




Aligned high shape: (271, 0)
Aligned low  shape: (271, 0)
Number of dates with high < low: 0

Clean return matrix shape: (270, 4)
Clean date range: 2020-01-02 00:00:00 to 2020-12-31 00:00:00
Assets: ['ADA-USD', 'BTC-USD', 'ETH-USD', 'LINK-USD']


In [17]:
# Cell 4: summarise performance from a daily return series

def summarize_performance(port_ret, name="strategy"):
    port_ret = port_ret.dropna()
    n = len(port_ret)
    if n == 0:
        print(f"{name}: no data")
        return {}

    mean_daily = port_ret.mean()
    vol_daily  = port_ret.std(ddof=1)
    sharpe_daily = mean_daily / vol_daily if vol_daily != 0 else np.nan

    mean_annual   = mean_daily * 252
    vol_annual    = vol_daily * np.sqrt(252)
    sharpe_annual = sharpe_daily * np.sqrt(252)

    # t-stat for mean > 0
    t_stat = mean_daily / (vol_daily / np.sqrt(n)) if vol_daily != 0 else np.nan

    # Drawdown and duration
    equity = (1 + port_ret).cumprod()
    running_max = equity.cummax()
    drawdown = equity / running_max - 1.0
    max_dd = drawdown.min()

    underwater = equity < running_max
    max_duration = 0
    current = 0
    for flag in underwater:
        if flag:
            current += 1
            max_duration = max(max_duration, current)
        else:
            current = 0

    stats = {
        "name": name,
        "n_days": n,
        "mean_daily": mean_daily,
        "vol_daily": vol_daily,
        "sharpe_daily": sharpe_daily,
        "mean_annual": mean_annual,
        "vol_annual": vol_annual,
        "sharpe_annual": sharpe_annual,
        "t_stat": t_stat,
        "max_dd": max_dd,
        "max_dd_days": max_duration,
    }

    print(f"\n=== {name} ===")
    for k, v in stats.items():
        if k == "name":
            continue
        print(f"{k:15s}: {v}")
    return stats


In [19]:
# Cell 5: cross-sectional momentum backtest (with costs & date window)

def backtest_cs_mom(
    ret_df,
    lookback=20,
    n_long=2,
    n_short=2,
    cost_bps=0.001,   # 10 bps per unit of turnover
    start_date=None,
    end_date=None,
    name=None
):
    """
    Cross-sectional momentum: long n_long winners, short n_short losers.

    ret_df   : DataFrame of daily returns (rows: dates, cols: assets)
    lookback : number of days used to compute momentum (rolling sum)
    n_long   : number of top assets to long
    n_short  : number of bottom assets to short
    cost_bps : proportional one-way transaction cost per unit of weight change
               e.g. 0.001 = 10 bps (0.10%) per unit of turnover
    start_date, end_date : date window for evaluation (out-of-sample)
                           earlier data are still used to compute momentum.
    """

    # ---- 1. Set evaluation window ----
    all_dates = ret_df.index
    if start_date is None:
        start_date = all_dates.min()
    if end_date is None:
        end_date = all_dates.max()

    # We need data up to end_date to compute signals
    sub = ret_df.loc[:end_date].copy()

    # ---- 2. Momentum signal: rolling sum over past 'lookback' days ----
    # Mom_t,i = sum_{k=1..lookback} r_{t-k,i}
    mom = sub.rolling(window=lookback).sum()

    assets = sub.columns
    n_assets = len(assets)

    # ---- 3. Raw target weights each day based on cross-sectional rank ----
    weights_raw = pd.DataFrame(index=sub.index, columns=assets, data=0.0)

    for dt in sub.index:
        sig = mom.loc[dt]

        if sig.isna().all():
            continue

        sig = sig.dropna()
        if len(sig) == 0:
            continue

        sig_sorted = sig.sort_values(ascending=False)

        n_long_eff  = min(n_long,  len(sig_sorted))
        n_short_eff = min(n_short, len(sig_sorted) - n_long_eff)

        long_assets  = sig_sorted.index[:n_long_eff]
        short_assets = sig_sorted.index[-n_short_eff:] if n_short_eff > 0 else []

        long_weight  =  0.5 / n_long_eff if n_long_eff > 0 else 0.0
        short_weight = -0.5 / n_short_eff if n_short_eff > 0 else 0.0

        for a in long_assets:
            weights_raw.loc[dt, a] = long_weight
        for a in short_assets:
            weights_raw.loc[dt, a] = short_weight

    # ---- 4. Transaction costs ----
    # Turnover_t = sum_i |w_raw_t,i - w_raw_{t-1,i}|
    weights_prev = weights_raw.shift(1)
    turnover = (weights_raw - weights_prev).abs().sum(axis=1).fillna(0.0)
    tc = cost_bps * turnover  # cost as a daily return

    # ---- 5. Portfolio returns (no look-ahead) ----
    # Use yesterday's weights on today's returns
    gross_ret = (weights_prev * sub).sum(axis=1)

    # Net return after transaction cost
    net_ret = gross_ret - tc

    # ---- 6. Restrict to evaluation window ----
    net_ret = net_ret.loc[start_date:end_date]

    if name is None:
        name = f"CS_MOM_L{lookback}_L{n_long}_S{n_short}"

    stats = summarize_performance(net_ret, name=name)
    return net_ret, stats


In [21]:
# Cell 6: define train / test windows

dates = ret_clean.index
n_dates = len(dates)

split_idx = int(n_dates * 0.7)  # 70% train, 30% test

train_start = dates[0]
train_end   = dates[split_idx]
test_start  = dates[split_idx + 1] if split_idx + 1 < n_dates else dates[-1]
test_end    = dates[-1]

print("Train window :", train_start, "to", train_end)
print("Test window  :", test_start,  "to", test_end)
print("Total days   :", n_dates, "(train:", split_idx + 1, ", test:", n_dates - (split_idx + 1), ")")


Train window : 2020-01-02 00:00:00 to 2020-10-10 00:00:00
Test window  : 2020-10-11 00:00:00 to 2020-12-31 00:00:00
Total days   : 270 (train: 190 , test: 80 )


In [27]:
# 🔁 NEW Cell 7: parameter grid search on TRAINING window only (fixed)

results_train = []

lookbacks = [5, 10, 20, 40]          # you can change these
long_short_pairs = [(1, 1), (2, 2)]  # 1 long/1 short vs 2 long/2 short

for L in lookbacks:
    for (nL, nS) in long_short_pairs:
        name = f"TRAIN_L{L}_Long{nL}_Short{nS}"
        net_ret_train, stats_train = backtest_cs_mom(
            ret_clean,
            lookback=L,
            n_long=nL,
            n_short=nS,
            cost_bps=0.001,          # 10 bps per unit turnover
            start_date=train_start,
            end_date=train_end,
            name=name
        )

        # 🔴 IMPORTANT: attach the parameters to the stats dict
        stats_train["lookback"] = L
        stats_train["n_long"]   = nL
        stats_train["n_short"]  = nS

        results_train.append(stats_train)

# Build DataFrame with both performance + parameter info
train_df = pd.DataFrame(results_train)

cols_show = [
    "name",
    "lookback",
    "n_long",
    "n_short",
    "sharpe_annual",
    "mean_annual",
    "vol_annual",
    "max_dd",
    "max_dd_days",
]

print("\n=== Training results (sorted by Sharpe) ===")
print(train_df[cols_show].sort_values("sharpe_annual", ascending=False))



/opt/anaconda3/lib/python3.12/site-packages/numpy/core/fromnumeric.py:59: RuntimeWarning: overflow encountered in accumulate
  return bound(*args, **kwds)



=== TRAIN_L5_Long1_Short1 ===
n_days         : 190
mean_daily     : 302.33103915351336
vol_daily      : 244.7980083408731
sharpe_daily   : 1.235022462815659
mean_annual    : 76187.42186668537
vol_annual     : 3886.047909083194
sharpe_annual  : 19.605373801132497
t_stat         : 17.023609837377577
max_dd         : -0.0010000000000000009
max_dd_days    : 1

=== TRAIN_L5_Long2_Short2 ===
n_days         : 190
mean_daily     : 190.59896359259758
vol_daily      : 176.45215177840615
sharpe_daily   : 1.0801736429485849
mean_annual    : 48030.93882533459
vol_annual     : 2801.0910714473175
sharpe_annual  : 17.1472249920518
t_stat         : 14.889166155126189
max_dd         : -0.09944395881562385
max_dd_days    : 64

=== TRAIN_L10_Long1_Short1 ===
n_days         : 190
mean_daily     : 289.18634932694664
vol_daily      : 247.27024081011436
sharpe_daily   : 1.1695153787188681
mean_annual    : 72874.96003039055
vol_annual     : 3925.293382863703
sharpe_annual  : 18.565481079333882
t_stat         

In [29]:
# 🔁 NEW Cell 8: choose best training config, then evaluate on TRAIN and TEST (fixed)

# 8.1 Pick best params based on training Sharpe
best_idx = train_df["sharpe_annual"].idxmax()
best_row = train_df.loc[best_idx]

best_lookback = int(best_row["lookback"])
best_n_long   = int(best_row["n_long"])
best_n_short  = int(best_row["n_short"])

print("\nBest training config:")
print("Row index:", best_idx)
print("Lookback :", best_lookback)
print("n_long   :", best_n_long)
print("n_short  :", best_n_short)
print("Train Sharpe:", best_row["sharpe_annual"])

# 8.2 Re-run on TRAIN window (for a clean summary)
ret_train_best, stats_train_best = backtest_cs_mom(
    ret_clean,
    lookback=best_lookback,
    n_long=best_n_long,
    n_short=best_n_short,
    cost_bps=0.001,
    start_date=train_start,
    end_date=train_end,
    name="BEST_TRAIN"
)

# 8.3 Run the SAME PARAMETERS on TEST window (OUT-OF-SAMPLE)
ret_test_best, stats_test_best = backtest_cs_mom(
    ret_clean,
    lookback=best_lookback,
    n_long=best_n_long,
    n_short=best_n_short,
    cost_bps=0.001,
    start_date=test_start,
    end_date=test_end,
    name="BEST_TEST"
)

# 8.4 Full-sample for reference (NOT used for tuning)
ret_full_best, stats_full_best = backtest_cs_mom(
    ret_clean,
    lookback=best_lookback,
    n_long=best_n_long,
    n_short=best_n_short,
    cost_bps=0.001,
    start_date=train_start,
    end_date=test_end,
    name="BEST_FULL"
)



Best training config:
Row index: 0
Lookback : 5
n_long   : 1
n_short  : 1
Train Sharpe: 19.605373801132497

=== BEST_TRAIN ===
n_days         : 190
mean_daily     : 302.33103915351336
vol_daily      : 244.7980083408731
sharpe_daily   : 1.235022462815659
mean_annual    : 76187.42186668537
vol_annual     : 3886.047909083194
sharpe_annual  : 19.605373801132497
t_stat         : 17.023609837377577
max_dd         : -0.0010000000000000009
max_dd_days    : 1

=== BEST_TEST ===
n_days         : 80
mean_daily     : 0.002245547278936355
vol_daily      : 0.02233007884394921
sharpe_daily   : 0.10056154725780701
mean_annual    : 0.5658779142919614
vol_annual     : 0.3544790122653258
sharpe_annual  : 1.5963650730001602
t_stat         : 0.8994498223640562
max_dd         : -0.11540536516715239
max_dd_days    : 24

=== BEST_FULL ===
n_days         : 270
mean_daily     : 212.75213734425873
vol_daily      : 247.45289629724957
sharpe_daily   : 0.8597682246915106
mean_annual    : 53613.538610753196
vol_ann

/opt/anaconda3/lib/python3.12/site-packages/numpy/core/fromnumeric.py:59: RuntimeWarning: overflow encountered in accumulate
  return bound(*args, **kwds)
/opt/anaconda3/lib/python3.12/site-packages/numpy/core/fromnumeric.py:59: RuntimeWarning: overflow encountered in accumulate
  return bound(*args, **kwds)
